In [1]:
import pyspark

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
10,application_1586389888165_0011,pyspark,idle,,,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
sparkSession = pyspark.sql.SparkSession(sc)
# How to Set up spark executors
conf = pyspark.SparkConf()
conf = conf.setMaster("local[*]").set('spark.executor.memory', "15G").set('spark.executor.cores', "3")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
hadoop_conf = sparkSession._jsc.hadoopConfiguration()
hadoop_conf.set("fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
# October Days
# 06/10/2015 --> Tuesday
# 18/10/2015 --> Sunday
# 22/10/2015 --> Thursday

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
# São Paulo Regions
# Central Regions: Bela Vista, Moema, Vila Mariana, Itaum Bibi
# Border regions: Brasilandia, Parelheiros, Carrão

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
column_names = "dt_server,dt_avl,line_id,latitude,longitude,id_avl,event,id_point".split(",")
# reading files
traces_0610 = sparkSession.read.option("header", "false").csv('s3a://mobility-traces-bus-sp/MO_15106.csv')
traces_1810 = sparkSession.read.option("header", "false").csv('s3a://mobility-traces-bus-sp/MO_151018.csv')
traces_2210 = sparkSession.read.option("header", "false").csv('s3a://mobility-traces-bus-sp/MO_151022.csv')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
# setting columns'names
for i in range(len(column_names)):
    traces_0610 = traces_0610.withColumnRenamed("_c%s" % i, column_names[i])
    traces_1810 = traces_1810.withColumnRenamed("_c%s" % i, column_names[i])
    traces_2210 = traces_2210.withColumnRenamed("_c%s" % i, column_names[i])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
# some statistics

print("The number of rows in 06/10/2015 is ",traces_0610.count())
print("The number of rows in 18/10/2015 is ",traces_1810.count())
print("The number of rows in 22/10/2015 is ",traces_2210.count())

In [8]:
# determining an hour of each row and compare if the date server is equal to bus date
from dateutil import parser
from pyspark.sql.functions import *# udf, struct, col, countDistinct
from pyspark.sql.types import StringType, IntegerType, FloatType, DateType, BooleanType

# 23:59, so returns 23
# receive a column
def get_hour(row):
    date = parser.parse(row)
    return date.hour

# compare if the server_date and avl_date are equal
def compare_hours(dt_server,dt_bus):

    date_server = parser.parse(dt_server)
    date_bus = parser.parse(dt_bus)
    
    return 1 if (date_server.year == date_bus.year) and (date_server.month == date_bus.month) and (date_server.day == date_bus.day) else 0

udf_get_hour = udf(get_hour, IntegerType())
udf_compare_hours = udf(compare_hours, IntegerType())

# Equal Dates Date server == date bus
traces_0610_equal_hours = traces_0610.withColumn("equal_dates", udf_compare_hours(traces_0610["dt_server"],traces_0610["dt_avl"]))
traces_0610_hours = traces_0610_equal_hours.withColumn("hour", udf_get_hour(traces_0610_equal_hours["dt_server"]))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
# adding neighborhood as a column
sc.install_pypi_package("geopandas")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Package already installed for current Spark context!
Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/context.py", line 1110, in install_pypi_package
    raise ValueError("Package already installed for current Spark context!")
ValueError: Package already installed for current Spark context!



In [22]:
import geopandas as gpd
sp = gpd.read_file('s3://mobility-traces-bus-sp/aux-files/shape-sp/DISTRITO_MUNICIPAL_SP_SMDUPolygon.shp')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
sc.install_pypi_package("matplotlib")

In [ ]:
sc.install_pypi_package("descartes")

In [30]:
len(sp)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

96

In [31]:
sc.install_pypi_package("shapely")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
# some statistics
# Checkig how many row have date server == date bus
# Filters example
#>>> df.filter(df.age > 3).collect()
#[Row(age=5, name=u'Bob')]
#>>> df.where(df.age == 2).collect()
#[Row(age=2, name=u'Alice')]

#>>> df.filter("age > 3").collect()
#[Row(age=5, name=u'Bob')]
#>>> df.where("age = 2").collect()
#[Row(age=2, name=u'Alice')]
#df = traces_0610_hours.filter(traces_0610_hours.equal_dates == True)

In [ ]:
#df.count()

In [ ]:
# df = traces_0610_hours.filter(traces_0610_hours.equal_dates == 0)

In [ ]:
#df.show()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

27527972